# Advanced techniques

Let's explore feature engineering techniques with the house prices dataset from Kaggle.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Advanced-techniques" data-toc-modified-id="Advanced-techniques-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Advanced techniques</a></span></li></ul></div>

In [4]:
import nbimporter
from src.dataset import Dataset
import warnings

"warnings.filterwarnings('ignore')"

houses = Dataset('./data/houseprices_prepared.csv.gz')
houses.describe()

Available types: [dtype('int64') dtype('O') dtype('float64')]
80 Features
43 categorical features
37 numerical features
16 categorical features with NAs
0 numerical features with NAs
64 Complete features
--
Target: Not set


We will replace the NA's in the dataset with 'None' or 'Unknown' since they're not really NA's. For no good reason the person in charge of encoding the file decided to assign NA's to values where the feature does not apply, but instead of using a value for that special condition (like the string 'None') he/she decided to use the actual NA.

In [5]:
houses.replace_na(column='Electrical', value='Unknown')
houses.replace_na(column=houses.names('categorical_na'), value='None')
houses.set_target('SalePrice')
houses.describe()

Available types: [dtype('int64') dtype('O') dtype('float64')]
80 Features
43 categorical features
37 numerical features
0 categorical features with NAs
0 numerical features with NAs
80 Complete features
--
Target: SalePrice


Split the dataset into train and test. Use default 20% split.

In [9]:
import featuretools as ft

es = ft.EntitySet()
es = es.entity_from_dataframe(entity_id='houses', 
                              dataframe=houses.data,
                             index='Id')
es

Entityset: None
  Entities:
    houses [Rows: 1460, Columns: 81]
  Relationships:
    No relationships

In [28]:
f_matrix, f_defs = ft.dfs(entityset=es,
                          target_entity='houses',
                          trans_primitives=['cum_count','cum_sum'],
                          max_depth=1)

In [29]:
f_matrix.shape

(1460, 80)

In [27]:
p = ft.primitives.list_primitives()
p[p['type'] == 'transform']

,name,type,description
19,diff,transform,Compute the difference between the value of a ...
20,or,transform,"For two boolean values, determine if one value..."
21,month,transform,Transform a Datetime feature into the month.
22,divide,transform,Creates a transform feature that divides two f...
23,cum_mean,transform,Calculates the mean of previous values of an i...
24,days_since,transform,"For each value of the base feature, compute th..."
25,week,transform,Transform a Datetime feature into the week.
26,subtract,transform,Creates a transform feature that subtracts two...
27,time_since_previous,transform,Compute the time since the previous instance.
28,is_null,transform,"For each value of base feature, return 'True' ..."
